In [2]:
import pickle

data_dict = pickle.load(open('/Users/joaoangnes/Documents/Faculdade/Sistema Inteligentes/Sign-Language-Detection-using-Distances-Python/data.pickle', 'rb'))

In [3]:
# =============== OBTER OS DADOS DO DADASET ===================
import numpy as np
import pandas as pd
import pprint
# Transformando os dados do dadaset e transformando em dataframe
# Convertendo em DataFrame
df_data = pd.DataFrame()

# # Transformando cada dados dos arrays em colunas do dataframe
for lista in data_dict['data']:
    linha_list = np.asarray(lista).reshape(-1)
    linha = pd.DataFrame([linha_list])
    df_data = pd.concat([df_data, linha], ignore_index=True)

classes_list = np.asarray(data_dict['class']) # Coleta as classes 
df_classes = pd.DataFrame(classes_list)

df_data.insert(loc=df_data.columns.get_loc(0), column='class', value=df_classes)

new_cols = ['distance_0', 'distance_1', 'distance_2', 'distance_3', 'distance_4', 'distance_5']

# Renomeia as colunas do dataframe com as novas colunas coletadas
df_data.columns = ['class'] + new_cols
df_data

,class,distance_0,distance_1,distance_2,distance_3,distance_4,distance_5
0,18,151.267974,68.447060,174.183811,16.278821,56.920998,96.519428
1,18,117.800679,71.217975,154.379403,36.235342,48.020829,102.415819
2,18,125.924581,65.368188,162.388423,25.612497,50.000000,99.368003
3,18,109.658561,66.940272,120.149906,29.427878,60.728906,112.294256
4,18,102.800778,76.537572,118.152444,46.818800,49.477268,99.005050
...,...,...,...,...,...,...,...
5709,14,112.605506,17.888544,171.842952,28.178006,34.365681,106.212052
5710,14,103.315052,16.155494,141.315250,19.104973,53.338541,91.678787
5711,14,108.452755,18.601075,167.451486,23.409400,28.017851,93.813645
5712,14,86.005814,14.212670,119.251834,15.556349,51.088159,86.000000


In [4]:
# ============= BALANCEAMENTO =================

# Como o dataset foi criado de forma balanceada o dataset não precisa ser tratado, pois já está balanceado
df_data['class'].sort_values().value_counts()

18    300
19    300
16    300
8     299
1     297
13    296
6     295
4     295
17    293
9     291
14    291
5     289
0     282
7     280
10    277
12    274
3     272
11    267
15    259
2     257
Name: class, dtype: int64

In [5]:
# Balancear os dados
from imblearn.over_sampling import SMOTE

# Segmentar os dados
df_atributos = df_data.drop(columns=['class'])
df_classes = df_data['class']

# dados_atributos.columns

resampler = SMOTE()
df_atributos, df_classes = resampler.fit_resample(df_atributos, df_classes)

df_classes.value_counts()

18    300
9     300
5     300
13    300
2     300
12    300
3     300
15    300
4     300
8     300
19    300
10    300
1     300
17    300
6     300
16    300
7     300
11    300
0     300
14    300
Name: class, dtype: int64

In [6]:
# ========= NORMALIZCÃO DOS DADOS ========================
from sklearn import preprocessing 
normalizer = preprocessing.MinMaxScaler()

# df_atributos = df_data.drop(columns=['class'])
# df_classes = df_data['class']

# Modelo para normalizar os dados númericos
normalization_model = normalizer.fit(df_atributos) # Não normalizar as classes - drop da coluba class antes desta linha

from pickle import dump
dump(normalization_model, open('/Users/joaoangnes/Documents/Faculdade/Sistema Inteligentes/Sign-Language-Detection-using-Distances-Python/normalization.model', 'wb'))

dt_num_normalized = normalization_model.fit_transform(df_atributos)

df_atributos = pd.DataFrame(dt_num_normalized, columns=df_atributos.columns)

df_data_normalized = df_atributos.join(df_classes, how = 'left')

df_atributos

,distance_0,distance_1,distance_2,distance_3,distance_4,distance_5
0,0.540066,0.295239,0.658706,0.074283,0.181344,0.312422
1,0.419452,0.307191,0.583812,0.165348,0.151411,0.334239
2,0.448730,0.281959,0.614100,0.116874,0.158067,0.322962
3,0.390108,0.288740,0.454367,0.134284,0.194150,0.370790
4,0.365393,0.330136,0.446814,0.213642,0.156309,0.321619
...,...,...,...,...,...,...
5995,0.116709,0.083388,0.022007,0.086341,0.353048,0.294209
5996,0.213020,0.094737,0.057837,0.137368,0.360523,0.246607
5997,0.172937,0.153283,0.088537,0.190368,0.400962,0.458790
5998,0.131731,0.112892,0.081679,0.143919,0.070311,0.296179


In [15]:
########################
# - HIPERPARAMETERS 
# Usando Random Forest

from sklearn.ensemble import RandomForestClassifier
from pprint import pprint
rf = RandomForestClassifier(random_state = 42)

pprint(rf.get_params())

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': 'sqrt',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_jobs': None,
 'oob_score': False,
 'random_state': 42,
 'verbose': 0,
 'warm_start': False}


In [16]:
import numpy as np

# Determinar as faixas de valores para cada hiperparemtro
n_estimators = [int(x) for x in np.linspace(start=10, stop=300, num=10)]
criterion = ['gini', 'entropy']
min_samples_split = [int(x) for x in np.linspace(start=2, stop=10, num=3)]
max_depth = [int(x) for x in np.linspace(start=10, stop=100, num=10)]
max_features = ['sqrt', 'log2']

# Criar a grade de valores
rf_grid = {
    'n_estimators' : n_estimators,
    'criterion' : criterion,
    'min_samples_split' : min_samples_split,
    'max_depth' : max_depth,
    'max_features' : max_features,
    'random_state': [42]
}

pprint(rf_grid)

{'criterion': ['gini', 'entropy'],
 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100],
 'max_features': ['sqrt', 'log2'],
 'min_samples_split': [2, 6, 10],
 'n_estimators': [10, 42, 74, 106, 138, 171, 203, 235, 267, 300],
 'random_state': [42]}


In [17]:
# Rodar o GridSearch (treinamento e avaliação do grid)
from sklearn.model_selection import RandomizedSearchCV

# rf = RandomForestClassifier()
rf_hyperparameters = RandomizedSearchCV(
                                        estimator = rf,
                                        param_distributions = rf_grid,
                                        n_iter = 200,
                                        cv = 10,
                                        verbose = 1,
                                        random_state = 42,
                                        n_jobs = -1
                                        )
df_atributos = df_data_normalized.drop(columns=['class'])
df_classes = df_data_normalized['class']

# rf_hyperparameters.fit(df_atributos, df_classes.values.ravel())
rf_hyperparameters.fit(df_atributos, df_classes)
pprint(rf_hyperparameters.best_params_)


Fitting 10 folds for each of 200 candidates, totalling 2000 fits
{'criterion': 'entropy',
 'max_depth': 60,
 'max_features': 'log2',
 'min_samples_split': 2,
 'n_estimators': 203,
 'random_state': 42}


In [25]:
df_atributos

,distance_0,distance_1,distance_2,distance_3,distance_4,distance_5
0,0.540066,0.295239,0.658706,0.074283,0.181344,0.312422
1,0.419452,0.307191,0.583812,0.165348,0.151411,0.334239
2,0.448730,0.281959,0.614100,0.116874,0.158067,0.322962
3,0.390108,0.288740,0.454367,0.134284,0.194150,0.370790
4,0.365393,0.330136,0.446814,0.213642,0.156309,0.321619
...,...,...,...,...,...,...
5995,0.208025,0.124088,0.065653,0.139970,0.388278,0.490048
5996,0.198322,0.093263,0.096596,0.309252,0.235233,0.420959
5997,0.172163,0.116639,0.081284,0.180008,0.092345,0.266030
5998,0.222929,0.115851,0.059725,0.146753,0.408746,0.473871


In [18]:
# ============== TREINAMENTO =================
# Treinar o modelo de ML

# print(df_classes.values.ravel())

rf = RandomForestClassifier(**rf_hyperparameters.best_params_)
# fertility_rf = rf.fit(df_atributos, df_classes.values.ravel())
fertility_rf = rf.fit(df_atributos, df_classes)
pprint(fertility_rf)

RandomForestClassifier(criterion='entropy', max_depth=60, max_features='log2',
                       n_estimators=203, random_state=42)


In [19]:
from sklearn.model_selection import cross_validate

scoring = ['precision_macro', 'recall_macro']
scores_cross = cross_validate(rf, df_atributos, df_classes, cv=10, scoring = scoring) # Rodar Cross Validation
print('Matriz de sensibilidades:', scores_cross['test_precision_macro'])
print('Matriz de especificidades:', scores_cross['test_recall_macro'])

# Metricas finais
print('Especificade:', scores_cross['test_precision_macro'].mean())
print('Sensibilidades:', scores_cross['test_recall_macro'].mean())

Matriz de sensibilidades: [0.94616608 0.95826131 0.95845187 0.95709091 0.97086009 0.96361277
 0.96649616 0.96683251 0.96011046 0.98264877]
Matriz de especificidades: [0.945      0.95666667 0.95666667 0.95333333 0.97       0.96166667
 0.965      0.96666667 0.95833333 0.98166667]
Especificade: 0.9630530926937464
Sensibilidades: 0.9615


In [20]:
import pickle

f = open('model.p', 'wb')
pickle.dump({'model': fertility_rf}, f)
f.close()

In [21]:
df_data_denormalized = pd.DataFrame( normalization_model.inverse_transform(df_atributos), columns=df_atributos.columns)
df_data_denormalized

,distance_0,distance_1,distance_2,distance_3,distance_4,distance_5
0,151.267974,68.447060,174.183811,16.278821,56.920998,96.519428
1,117.800679,71.217975,154.379403,36.235342,48.020829,102.415819
2,125.924581,65.368188,162.388423,25.612497,50.000000,99.368003
3,109.658561,66.940272,120.149906,29.427878,60.728906,112.294256
4,102.800778,76.537572,118.152444,46.818800,49.477268,99.005050
...,...,...,...,...,...,...
5995,59.135484,28.768089,17.360893,30.673787,118.451052,144.525503
5996,56.443068,21.621842,25.543329,67.771307,72.944415,125.853150
5997,49.184816,27.041157,21.494256,39.448109,30.457911,83.981372
5998,63.271052,26.858380,15.793307,32.160302,124.537223,140.153464
